# How to Transform Time Series to a Supervised Learning Problem

Time series forecasting can be framed as a supervised learning problem. This re-framing of your
time series data allows you access to the suite of standard linear and nonlinear machine learning
algorithms on your problem. 

In [1]:
import numpy as np

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
s = np.sin(np.linspace(0,5))
X, y = split_sequence(s, 4)

In [9]:
# Define univariate time series
series = np.array([i+1 for i in range(10)])
print(series.shape)

# Transform into a supervised learning problem
X, y = split_sequence(series, 3)
print(X.shape, y.shape)

# Transform input from [samples, features] to [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X.shape)

(10,)
(7, 3) (7,)
(7, 3, 1)


## Data Preparation Example